###Scraping procedure to import useful infos about apartment rental listings in Milan from **Subito.it**:

In [ ]:
import requests
import webbrowser
from bs4 import BeautifulSoup
from pprint import pprint
from time import sleep
import pandas as pd
import itertools as it
import json
import re
import sys

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
sys.path.append('/content/drive/MyDrive/Progetto Data Management/Code')
from functions import *

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
pd.set_option("display.max_colwidth", None)

In [ ]:
# Let's select the attributes cited below from the pages.
# We have empty lists which we are going to fill next.

titles = []
prices = []
areas = []
locals = []
bathrooms = []
floor = []
urls = []
agenzia = []


# Let's go scraping the (50) available pages into which the total number of ads of interest is divided (updated 3 June 2023)

headers = ({'User-Agent':
              'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36'})

for page in range(1, 50):
  link = "https://www.subito.it/annunci-lombardia/affitto/appartamenti/milano/milano/?o=" + str(page)
  r = requests.get(link, headers=headers)
  page_html = BeautifulSoup(r.text, 'html.parser')
  house_containers = page_html.find_all('div', class_="items__item item-card item-card--big BigCard-module_card__Exzqv")

  if house_containers != []:
    for container in house_containers:
      if container.find_all("div", class_ = "CardImage-module_count__HEjKn") != []: # consider only listings with thumbnails
  
        # Price
        try:            
          price = container.find_all('p')[0].text
          price_ = [int(price[s]) for s in range(0, len(price)) if price[s].isdigit()]
          price = ''
          for x in price_:
            price = price+str(x)
          prices.append(int(price))
        except:
          prices.append(None)
      


        # Area
        try:
          m2 = container.find_all('p')[1].text
          m2_ = [int(m2[s]) for s in range(0,len(m2)) if m2[s].isdigit()]
          m2 = ''
          for x in m2_:
            m2 = m2+str(x)
          areas.append(int(m2))
        except:
          areas.append(None)


        # Locals       
        try:     
          locali = container.find_all('p')[2].text
          locali_ = [int(locali[s]) for s in range(0,len(locali)) if locali[s].isdigit()]
          locali = ''
          for x in locali_:
            locali = locali+str(x)
          locals.append(int(locali))
        except:
          locals.append(None)


        # Floor
        try:            
          piano = container.find_all('p')[3].text

          if (piano == "Piano T") | (piano == "Rialz."):
            floor.append(0)

          elif (piano == "Semint.") | (piano == "Interr."):
            floor.append(-1)

          else:
            piano_ = [int(piano[s]) for s in range(0,len(piano)) if piano[s].isdigit()]
            piano = ''
            for x in piano_:
              piano = piano+str(x)
            floor.append(int(piano))
        except:
          floor.append(None)


        # Bathrooms 
        try:
          bagni = container.find_all('p')[4].text
          bagni_ = [int(bagni[s]) for s in range(0,len(bagni)) if bagni[s].isdigit()]
          bagni = ''
          for x in bagni_:
            bagni = bagni+str(x)
          bathrooms.append(int(bagni))
        except:
          bathrooms.append(None)


        # Title
        name = container.find_all('h2')[0].text
        titles.append(name)


        # URL
        link = container.find_all('a')[0].get('href')
        urls.append(link)


        # Agency
        agency = container('span', class_="index-module_sbt-text-atom__ed5J9 index-module_token-overline__ESoEk index-module_size-small__XFVFl index-module_badge__LFSGS index-module_shop-type__iKbhv")
        if agency != []:
          agenzia.append("agency")
        else:
          agenzia.append("private")

  else:
    break
    
  sleep(1)

In [ ]:
# We then obtain the descriptions of each ad via the link obtained above. 
# These are in fact located within the web page of each ad and not on the main page.
# Same for the real estate agency addresses, if any, which will be useful later on

descriptions = []
agency_address = []

for link in urls: 
  r = requests.get(link, headers=headers)
  page_html = BeautifulSoup(r.text, 'html.parser')
  container_desc = page_html.find_all('p', class_="index-module_sbt-text-atom__ed5J9 index-module_token-body__GXE1P size-normal index-module_weight-book__WdOfA AdDescription_description__gUbvH index-module_preserve-new-lines__ZOcGy")
  container_addr = page_html.find_all('p', class_="index-module_sbt-text-atom__ed5J9 index-module_token-body__GXE1P size-normal index-module_weight-book__WdOfA index-module_body_text__v5hiP")

  if container_desc != []:
    descriptions.append(container_desc[0].text)
  
  else:
    descriptions.append(None)
  
  if container_addr != []:       
    agency_address.append(container_addr[0].text)
  
  else:
    agency_address.append(None)


In [ ]:
cols = ['Title', 'Price/month', 'Size (m^2)', 'Locals', 'Bathrooms', 'Floor', 'Description', 'If_agency', 'Agency_address', 'URL']

subito = pd.DataFrame({'Title': titles,
                        'Price/month': prices,
                        'Size (m^2)': areas,
                        'Locals': locals,
                        'Bathrooms': bathrooms,
                        'Floor': floor,
                        'Description': descriptions,
                        'If_agency': agenzia,
                        'Agency_address': agency_address,
                        'URL': urls,})[cols]

In [ ]:
# Import of street-to-district json file
with open('/content/drive/MyDrive/Progetto Data Management/Data/quartieri.json', 'r') as file:
    quartieri = json.load(file)

In [ ]:
diz_address = addressFinder2(quartieri, subito['Description'] + ' ' + subito['Title'], subito['Agency_address'])
subito = subito.reset_index()
subito['Address'] = subito['index'].astype(int).map(diz_address)
subito = subito.drop('index', axis=1) 

#title_desc = (subito['Description'] + ' ' + subito['Title'])
#streets = addressFinder(quartieri, title_desc, subito["Agency_address"])
#subito["Address"] = streets

In [ ]:
districts = address_to_district(subito, quartieri)
subito["District"] = districts

In [ ]:
subito.head(15)

,Title,Price/month,Size (m^2),Locals,Bathrooms,Floor,Description,If_agency,Agency_address,URL,Address,District
0,DA SUBITO bilocale 50 mq con balcone - 600 euro,600.0,40,2.0,1.0,1.0,"SI AFFITTA DA SUBITO GRANDE APPARTAMENTO DI 50 MQ A ROSATE.\nIN OTTIME CONDIZIONI E BEN ARREDATO, È COMPOSTO DA INGRESSO SU AMPIO SOGGIORNO, ZONA RELAX CON DIVANO E BALCONE, ANGOLO COTTURA COMPLETO DI ELETTRODOMESTICI, CAMERA MATRIMONIALE CON ARMADIO, COMODINI E SPECCHIO.\nSI TROVA AL PRIMO E ULTIMO PIANO DELLA PALAZZINA. CONTESTO PRIVATO E TRANQUILLO CON INGRESSO RISERVATO.\nIL RISCALDAMENTO È DI TIPO AUTONOMO, OTTIMALE PER IL CONTROLLO MODERATO DEI CONSUMI.\n\nRICHIESTA MENSILE EURO 600,00 CON LE SOLE UTENZE A PARTE - LIBERO E VISIONABILE DA SUBITO, CHIAMARE AL 3792766678 oppure 3517908666\n\nRIF. 29515\n\nAffitto Privato non è un'agenzia immobiliare e non svolge attività di mediazione. E' un inserzionista che offre un servizio informativo di accesso ad una banca dati, contenente offerte di immobili di proprietari privati reperibili gratuitamente anche su altri portali.\nNon ci sono costi di mediazione ma un'unica quota per usufruire del servizio.",agency,None,https://www.subito.it/appartamenti/da-subito-bilocale-50-mq-con-balcone-600-euro-milano-496027756.htm,,
1,"MM Loreto, bilocale ristrutturato e condizionato",1400.0,70,2.0,1.0,7.0,"Con affaccio sui giardini Polotti, il parco di Piazza Aspromonte, ampio bilocale panoramico sito all'ottavo piano di 70 mq composto da ingresso, ampia camera con balcone, soggiorno con angolo cottura, bagno finestrato, doppio ripostiglio, camera cabina armadio attrezzata. Splendida vista sulla skyline milanese. Recentemente ristrutturato con grande cura e dettagli di pregio come la vasca con le zampe di leone, piano cottura a induzione, aria condizionata, sedie magis di design. Arredato per 2 persone. Molto luminoso, pavimenti in marmo, porta blindata, lavatrice. Libero il 19 settembre.\nElegante stabile signorile con portierato al mattino, ascensore, videosorveglianza h24 e locale comune biciclette.\nLa metropolitana Loreto verde e rossa è a pochi metri. Nelle immediate vicinanze autobus 39, 55, 56, 60, 62, 81, 90, 91, 92, 965 e tram 1. BikeMi di fronte allo stabile.\nSupermercato a meno di 50 metri, due minuti a piedi per Corso Buenos Aires, una delle arterie dello shopping milanese e cinque minuti a piedi per il Politecnico. Ottime condizioni interne. \nC.e. E 161,22 kw/m2a. Prezzo € 1400 incluse spese condominiali, acqua e riscaldamento centralizzato. Privato affitta con permanenza minima 1 anno. Tel. 3338427607 e 0229400368.",private,None,https://www.subito.it/appartamenti/mm-loreto-bilocale-ristrutturato-e-condizionato-milano-496025309.htm,corso buenos aires,buenos-aires-venezia
2,Trilocale P. SPOSI 1600eu SPESE COMPRESE,1600.0,80,3.0,1.0,5.0,"Adiacente a scuole di diverso ordine e grado\n\nIn LARGO PROMESSI SPOSI , nei pressi della zona FAMAGOSTA , \n\nsi affitta TRILOCALE di circa 80 mq, composto da camera da letto, due ampie camere da letto e il servizio.\n\nGrazie alla presenza di DUE BALCONI, la soluzione risulta essere MOLTO LUMINOSA. \n\nOttima è la sua posizione comoda a tutti i servizi, scuole, piscine, palestre, negozi di ogni genere.\n\nCompleta la proprietà la CANTINA al piano interrato.\n\nL'appartamento risulta COMPLETAMENTE ARREDATO e dotato di ogni confort, ristrutturato di recente e si presenta in ottime condizioni interne, \nIl palazzo è abitato bene , decoroso con ascensore, precisamente al QUINTO piano, luminoso con esposizione doppia.\n\nRICHIESTA DI EURO 1600\n\nTUTTO COMPRESO!!!\n\nSPESE COMPRESE!!!\n\nTUTTO RISTRUTTURATO A NUOVO!!!\n\nAMPIA METRATURA!!! 80 mq!!!\n\nMOLTO LUMINOSO!!!\n\nAffitto privato non è un'agenzia immobiliare e non svolge attività di mediazione. E' un'inserzionista che offre un servizio\ninformativo di accesso ad una banca dati, contenente offerte di immobili di proprietari privati reperibili gratuitamente\nsu altri portali... non ci sono costi di mediazione ma u

In [ ]:
len(subito) # 1123 from the observations of the first row dataset

1123

##Data Cleaning

### Let's delete the special characters *\t*, *\n* and *\r* from the descriptions:

In [ ]:
for i in range(0, len(descriptions)):
  descriptions[i] = re.sub(r'[\r\n\t]+', ' ', descriptions[i])

subito["Description"] = descriptions

###Drop the duplicates:

In [ ]:
# We first select all observations in the dataset that have the variables in question in common, 
# the keep=False option allows all rows to be kept, otherwise a smaller subset would have been considered 
# a smaller subset where the first or last observation for each combination would have been excluded 
# (keep = 'first' or 'last').
dup = subito[subito.duplicated(subset = ['Price/month', 'Size (m^2)', 'Locals', 'Bathrooms', 'Floor', 'If_agency', 'Agency_address', 'District'], keep=False)]

# Of these we consider all those with a not null description.
dup = dup[dup["Description"].notnull()]

len(dup) # we get a total of 191 rows to start working on

191

In [ ]:
# For each combination of the above variables, we display the relative indices of the starting dataset. 
# It would be too computationally onerous to compare one row with all the others each time, so we compare the
# descriptions only within the created groups, which already have the same attributes

groups = dup.groupby(by = ['Price/month', 'Size (m^2)', 'Locals', 'Bathrooms', 'Floor', 'If_agency', 'Agency_address', 'District'], sort = False).groups

In [ ]:
# We create a new variable in which we insert the descriptions converted to strings, 
# this to make the work easier because of the format.
desc = dup["Description"].astype(str) 


# We remove the punctuation within each description (replacing it with a space)
for i in desc.index: 
  desc[i] = re.sub(r'[^\w]+', ' ', desc[i])


# Then we convert all words to lower case and split them
desc = desc.str.lower().str.split() 


# Finally we convert everything to set format (a collection of unique values), 
# wanting to compare descriptions on the basis of words taken once.
# It suffices just that a single word (e.g. conjunction) is repeated as many times in one description 
# and less in the other to increase the percentage of words in common between the two.
for i in desc.index: 
    desc[i] = set(desc[i])


In [ ]:
lista = [] 
for val in groups.values():
  lista.append(desc.loc[val])

In [ ]:
# We create an empty list into which we insert the indexes of the rows of the dataset that share at least 90% of the words
final_dup = [] 

# In the loop, all possible combinations without repetition (itertools library, combinations command) of
# two sets of words contained within a group, which is iterated each time.
for i in range(0, len(lista)):
    for x, y in it.combinations(lista[i], 2):
        if len(x & y)/len(x.union(y)) >= 0.9:
            final_dup.append(lista[i][lista[i] == x].index[0])
            final_dup.append(lista[i][lista[i] == y].index[0])


In [ ]:
# We obtain the duplicates of the original dataset, eliminating the repetitions present
# within the final_dup list and occurring when there are two or more duplicate descriptions within
# of the same group.
new = subito.iloc[final_dup].drop_duplicates() 
len(new)

52

In [ ]:
subito2 = subito.drop(index=new.index).reset_index(drop=True)
len(subito2) # total of 1071 advertisements (05/06/2023)

1071

###Drop the price and size outliers:

In [ ]:
subito2.describe()

,Price/month,Size (m^2),Locals,Bathrooms,Floor
count,1010.000000,1071.000000,1053.000000,868.000000,949.000000
mean,2011.491089,60.787115,2.096866,1.109447,2.354057
std,15704.013091,25.744276,0.814624,0.326815,1.944113
min,40.000000,1.000000,1.000000,1.000000,-1.000000
25%,850.000000,45.000000,2.000000,1.000000,1.000000
50%,1150.000000,55.000000,2.000000,1.000000,2.000000
75%,1400.000000,70.000000,2.000000,1.000000,4.000000
max,430000.000000,190.000000,6.000000,3.000000,7.000000


In [ ]:
# - set a minimum of sqm (national disposition for a single local apartment)
# - set a minimum and maximum for price/month variable in order to
# drop the observations related to the short-rentals listings

outliers = (subito2[(subito2["Size (m^2)"] < 28) |
            (subito2["Price/month"] < 300)|
            (subito2["Price/month"] > 10000)]).index

len(outliers)

78

In [ ]:
subito3 = subito2.drop(index=outliers).reset_index(drop=True)

len(subito3) # 957 final total observations (updated on 05/06/2023)

993

In [ ]:
subito3[["Price/month", "Size (m^2)"]].describe()

,Price/month,Size (m^2)
count,938.000000,993.000000
mean,1257.681237,63.021148
std,529.516972,24.683287
min,350.000000,28.000000
25%,900.000000,45.000000
50%,1200.000000,59.000000
75%,1412.750000,70.000000
max,5000.000000,190.000000


###Export in JSON

In [ ]:
with open("/content/drive/MyDrive/Progetto Data Management/Data/subito.json", "w") as file:
  json.dump(subito3.to_dict(orient="index"), file)